In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset = pd.read_csv("Social_Network_Ads.csv")
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [3]:
dataset = pd.get_dummies(dataset,dtype=int, drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [4]:
dataset = dataset.drop("User ID", axis=1)
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [5]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [6]:
independent = dataset[["Age","EstimatedSalary","Gender_Male"]]
dependent = dataset["Purchased"]

In [7]:
independent.shape

(400, 3)

In [8]:
dependent

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [9]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(independent, dependent, test_size = 1/3, random_state = 0)

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

#Here the fit.transform function Standardize the Undefined data to Defined Data and make it to be saved in X_train
X_train = sc.fit_transform(X_train)

#Here the transform function uses the process occurred in fit.transform function for the input X_test and make it to be saved in X_train
X_test = sc.transform(X_test)

In [27]:
from sklearn.model_selection import GridSearchCV #Here cv refers to Cross Validation
from sklearn.tree import DecisionTreeClassifier

#Creating a Dictionary for Playable Parameters
param_grid = {'criterion':['gini','entropy'], 'max_features': ['auto','sqrt','log2'], 'splitter':['best','random']} 

#if refit = false then final model will be considered as Last Combination of parameter_grid like (linear,3000,scale)
#n_jobs = -1 refers to Run all the Models Parallely
grid = GridSearchCV(DecisionTreeClassifier(), param_grid, refit = True, verbose = 3, n_jobs=-1, scoring='f1_weighted')

#All the above steps are called "Procedure Load"

#Model Creation
grid.fit(X_train,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Anaconda\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
20 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
7 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Anaconda\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Anaconda\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\Anaconda\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "C:\Anaconda\Lib\site-packages\sklearn\utils\_param_validation.py", line 95, in validate_paramete

GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [29]:
# print best parameter after tuning 
#print(grid.best_params_) 
results = grid.cv_results_
#print(re)

In [31]:
y_pred = grid.predict(X_test)

In [33]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

[[78  7]
 [ 4 45]]


In [35]:
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, y_pred)
print(clf_report)

              precision    recall  f1-score   support

           0       0.95      0.92      0.93        85
           1       0.87      0.92      0.89        49

    accuracy                           0.92       134
   macro avg       0.91      0.92      0.91       134
weighted avg       0.92      0.92      0.92       134



In [37]:
from sklearn.metrics import f1_score
f1_macro = f1_score(y_test,y_pred,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'criterion': 'entropy', 'max_features': 'sqrt', 'splitter': 'best'}: 0.9183922682195829


In [39]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.9180072028811523

In [41]:
#To make All the Results into Tabular Format

table = pd.DataFrame.from_dict(results)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002987,0.001342,0.000000,0.000000,gini,auto,best,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
1,0.001790,0.001289,0.000000,0.000000,gini,auto,random,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
2,0.005822,0.001899,0.005493,0.001525,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.804764,0.888107,0.814409,0.905069,0.886792,0.859828,0.041638,3
3,0.003128,0.000934,0.007472,0.000522,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.822092,0.813511,0.869709,0.833653,0.805463,0.828886,0.022452,5
4,0.002736,0.000680,0.005492,0.001107,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.847141,0.828959,0.833323,0.906166,0.905069,0.864132,0.034403,2
5,0.003040,0.000877,0.006605,0.001577,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.829615,0.815037,0.777291,0.847020,0.847020,0.823197,0.025885,6
6,0.001027,0.000060,0.000000,0.000000,entropy,auto,best,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
7,0.001645,0.001295,0.000000,0.000000,entropy,auto,random,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
8,0.003357,0.001002,0.007576,0.001141,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.826263,0.906252,0.833323,0.851527,0.923510,0.868175,0.039396,1
9,0.003273,0.001326,0.006897,0.000917,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.808927,0.866968,0.680964,0.849057,0.867097,0.814602,0.070109,7


In [43]:
age_input=float(input("Age:"))
salary_input=float(input("BMI:"))
sex_male_input=int(input("Sex Male 0 or 1:"))

Age: 30
BMI: 40
Sex Male 0 or 1: 1


In [45]:
#prediction
#Keyword - predict is function of Application 
Future_Prediction = grid.predict([[age_input,salary_input,sex_male_input]])
print("Future_Prediction{}:".format(Future_Prediction))

Future_Prediction[1]:


In [21]:
#dir(clf_report)

In [47]:
# Naming the Model with its extension .sav
import pickle
filename = "finalized_model_DT_Grid_classification_Assignment.sav"

#Model will be saved in respective path, once after the below Code Execution
# Keyword - wb refers to "write binary"
pickle.dump(grid,open(filename,'wb'))

In [49]:
# Load the Saved Model and check the same to send it for Phase-02 (Deployment)
# # Keyword - rb refers to "read binary"

loaded_model=pickle.load(open("finalized_model_DT_Grid_classification_Assignment.sav",'rb'))
result = loaded_model.predict([[age_input,salary_input,sex_male_input]]) 
result

array([1], dtype=int64)